In [ ]:
#######
# load data to pandas
# train
# tune parameters on holdout set
# test
# kfolds cross validation
#######
# comparisons:
# linear regression
# decision trees (random forest via scikit learn)
# ff neural network (lin, relu, lin) higher order polyn

In [ ]:
%load_ext autoreload
%autoreload 2

# Imports
import pandas as pd
from core_insure.assessor.home_assessor import HomeAssessor
from ruamel.yaml import YAML
import numpy as np
import math
from torch import nn

LABEL_KEY = 'claimAmount'

In [242]:
# Processing and training methods
def value_of_interest(value):
    if pd.isna(value):
        return False
    return True

def get_row_xy(row):
    y_label = row[LABEL_KEY] if value_of_interest(row[LABEL_KEY]) else 0
    row = row.drop([LABEL_KEY])
    rowiter = row.iteritems()
    x_array = [item[1] if value_of_interest(item[1]) else 0 for item in rowiter]
    return x_array, y_label

def get_all_xy(df):
    all_x = []
    all_y = []
    for row in df.iterrows():
        row_contents = row[1]
        x, y = get_row_xy(row_contents)
        all_x.append(x)
        all_y.append([y])
    return all_x, all_y

def calculate_train_val_loss(curr_assessor, train_x, train_y, test_x, test_y):
    # exploding gradients?
    train_output = curr_assessor.train(train_x, train_y)

    # Train loss 
    model = curr_assessor.model
    train_pred_y = model.eval(train_x)
    loss = model.loss
    train_loss = loss(model._torch_var(train_pred_y), model._torch_var(train_y))
    print(f'Train Loss: {train_loss}')
    
    # Test linear regression
    test_y_pred = model.eval(test_x)
    test_loss = loss(model._torch_var(test_y_pred), model._torch_var(test_y))
    print(f'Test Losss: {test_loss}')
    
    return train_loss, test_loss

def train_test_split(curr_dataframe, train_percent, test_percent):
    total_rows = curr_dataframe.shape[0]
    print(f'Total test set: {total_rows}')
    train_size = math.floor(total_rows*(train_percent/100))
    test_size = total_rows - train_size
    print(f'Training set: {train_size}, Testing set: {test_size}')
    train_x, train_y = get_all_xy(curr_dataframe.head(train_size))
    print(f'Train set created')
    test_x, test_y = get_all_xy(curr_dataframe.tail(test_size))
    print(f'Test set created')
    return {
        'train_x': train_x,
        'train_y': train_y,
        'test_x': test_x,
        'test_y': test_y
    }

# def train_with_config(linreg_config, df_data):
#     epochs = linreg_config.get('epochs')
#     epoch_loss = 0
#     for epoch in range(epochs):
#         for row in df_data.iterrows():
#             row_contents = row[1]
#             x, y = get_row_xy(row_contents)
#             
#             y_pred = self.model(self._torch_var(x))
#             loss = self.loss(y_pred, self._torch_var(y))
#             epoch_loss = loss.data[0]
#     
#             self.optimizer.zero_grad()
#             loss.backward()
#             self.optimizer.step()
#         print(f'Epoch {epoch} loss: {epoch_loss}')

# train_with_config(linreg_config, sorted_numerical_df)

In [ ]:
# Load data to pandas
# This dataset was downloaded from: 
# https://www.fema.gov/openfema-dataset-individual-assistance-housing-registrants-large-disasters-v1
file = open('core_insure/data/IndividualAssistanceHousingRegistrantsLargeDisasters.csv', 'r')
pd_data = pd.read_csv(file)

In [ ]:
# Get config
default_config_file = open('core_insure/config.yaml', 'r')
yaml = YAML()
config = yaml.load(default_config_file)
print(config)

In [ ]:
# Data processing
# Columns available
all_columns = list(pd_data.columns.values)

# calculate label: claimAmount based on repairs and replacements
pd_data = pd_data.fillna(0)
pd_data[LABEL_KEY] = pd_data['repairAmount'] + pd_data['replacementAmount']

# For now, filter on just numerical data
# Later, can convert string columns (embedding, bins, num categories)
only_numerical_data = pd_data._get_numeric_data()
sorted_numerical_df = pd_data.reindex(sorted(only_numerical_data.columns), axis=1)
numeric_columns = sorted(sorted_numerical_df.columns.values)
print(list(numeric_columns))

In [ ]:
# remove all data where repairs were not made
nonzero_numerical_df = sorted_numerical_df.replace(0, np.nan)
nonzero_numerical_df = nonzero_numerical_df[nonzero_numerical_df[LABEL_KEY].notnull()] 
len(nonzero_numerical_df)

In [243]:
# Feature Extraction
columns = [
    'roofDamageAmount',
    'foundationDamageAmount',
    'floodDamageAmount',
    'waterLevel',
    'destroyed',
    'habitabilityRepairsRequired',
    'rentalAssistanceAmount',
    'ppfvl',
    'rpfvl',
    'personalPropertyEligible',
    LABEL_KEY
]
final_sorted_df_nz = pd.DataFrame(nonzero_numerical_df, columns=columns)
final_sorted_df = pd.DataFrame(sorted_numerical_df, columns=columns)

final_sorted_df_nz = final_sorted_df_nz.replace(np.nan, 0)
final_sorted_df = final_sorted_df.replace(np.nan, 0)

In [244]:
# 80-20 Split
# full_dataset = final_sorted_df
full_dataset = final_sorted_df.head(10000)
train_test_dataset = train_test_split(full_dataset, 80, 20)

full_dataset_nz = final_sorted_df_nz.head(10000)
train_test_dataset_nz = train_test_split(full_dataset_nz, 80, 20)

Total test set: 10000
Training set: 8000, Testing set: 2000
Train set created
Test set created
Total test set: 10000
Training set: 8000, Testing set: 2000
Train set created
Test set created


In [246]:
# Train linear regression - ALL CLAIMS
assessor_config =  config['assessor']
assessor_config['linear_regression']['epochs'] = 1000
assessor = HomeAssessor(assessor_config)

calculate_train_val_loss(assessor, 
                         train_test_dataset['train_x'], 
                         train_test_dataset['train_y'],
                         train_test_dataset['test_x'],
                         train_test_dataset['test_y'])

Epoch 0, Loss: 335.83953857421875, y_pred preview: tensor([0.0164])
Epoch 1, Loss: 331.96307373046875, y_pred preview: tensor([0.0213])
Epoch 2, Loss: 328.9444580078125, y_pred preview: tensor([0.0255])
Epoch 3, Loss: 325.9828186035156, y_pred preview: tensor([0.0293])
Epoch 4, Loss: 323.0314025878906, y_pred preview: tensor([0.0328])
Epoch 5, Loss: 320.14190673828125, y_pred preview: tensor([0.0360])
Epoch 6, Loss: 317.33856201171875, y_pred preview: tensor([0.0389])
Epoch 7, Loss: 314.6184387207031, y_pred preview: tensor([0.0417])
Epoch 8, Loss: 312.0488586425781, y_pred preview: tensor([0.0441])
Epoch 9, Loss: 309.55133056640625, y_pred preview: tensor([0.0463])
Epoch 10, Loss: 307.0854187011719, y_pred preview: tensor([0.0481])
Epoch 11, Loss: 304.6532897949219, y_pred preview: tensor([0.0497])
Epoch 12, Loss: 302.23797607421875, y_pred preview: tensor([0.0510])
Epoch 13, Loss: 299.83074951171875, y_pred preview: tensor([0.0521])
Epoch 14, Loss: 297.4486083984375, y_pred preview: 

Epoch 128, Loss: 184.74685668945312, y_pred preview: tensor([-0.0167])
Epoch 129, Loss: 184.01829528808594, y_pred preview: tensor([-0.0168])
Epoch 130, Loss: 183.2878875732422, y_pred preview: tensor([-0.0168])
Epoch 131, Loss: 182.5835723876953, y_pred preview: tensor([-0.0170])
Epoch 132, Loss: 181.93748474121094, y_pred preview: tensor([-0.0171])
Epoch 133, Loss: 181.33029174804688, y_pred preview: tensor([-0.0172])
Epoch 134, Loss: 180.7571563720703, y_pred preview: tensor([-0.0174])
Epoch 135, Loss: 180.1817626953125, y_pred preview: tensor([-0.0174])
Epoch 136, Loss: 179.61802673339844, y_pred preview: tensor([-0.0175])
Epoch 137, Loss: 179.05337524414062, y_pred preview: tensor([-0.0175])
Epoch 138, Loss: 178.5302276611328, y_pred preview: tensor([-0.0175])
Epoch 139, Loss: 178.06968688964844, y_pred preview: tensor([-0.0176])
Epoch 140, Loss: 177.6300811767578, y_pred preview: tensor([-0.0177])
Epoch 141, Loss: 177.20550537109375, y_pred preview: tensor([-0.0179])
Epoch 142, L

Epoch 264, Loss: 151.37913513183594, y_pred preview: tensor([-0.0491])
Epoch 265, Loss: 151.16661071777344, y_pred preview: tensor([-0.0489])
Epoch 266, Loss: 150.94969177246094, y_pred preview: tensor([-0.0486])
Epoch 267, Loss: 150.7461395263672, y_pred preview: tensor([-0.0481])
Epoch 268, Loss: 150.53672790527344, y_pred preview: tensor([-0.0479])
Epoch 269, Loss: 150.31686401367188, y_pred preview: tensor([-0.0475])
Epoch 270, Loss: 150.09744262695312, y_pred preview: tensor([-0.0472])
Epoch 271, Loss: 149.88372802734375, y_pred preview: tensor([-0.0469])
Epoch 272, Loss: 149.6713409423828, y_pred preview: tensor([-0.0466])
Epoch 273, Loss: 149.4542694091797, y_pred preview: tensor([-0.0465])
Epoch 274, Loss: 149.25282287597656, y_pred preview: tensor([-0.0461])
Epoch 275, Loss: 149.03579711914062, y_pred preview: tensor([-0.0459])
Epoch 276, Loss: 148.8212127685547, y_pred preview: tensor([-0.0456])
Epoch 277, Loss: 148.59445190429688, y_pred preview: tensor([-0.0454])
Epoch 278,

Epoch 383, Loss: 131.5072479248047, y_pred preview: tensor([-0.0350])
Epoch 384, Loss: 131.48544311523438, y_pred preview: tensor([-0.0351])
Epoch 385, Loss: 131.45606994628906, y_pred preview: tensor([-0.0356])
Epoch 386, Loss: 131.40032958984375, y_pred preview: tensor([-0.0354])
Epoch 387, Loss: 131.30287170410156, y_pred preview: tensor([-0.0357])
Epoch 388, Loss: 131.2564239501953, y_pred preview: tensor([-0.0358])
Epoch 389, Loss: 131.35531616210938, y_pred preview: tensor([-0.0355])
Epoch 390, Loss: 131.31890869140625, y_pred preview: tensor([-0.0361])
Epoch 391, Loss: 131.1284637451172, y_pred preview: tensor([-0.0357])
Epoch 392, Loss: 131.05935668945312, y_pred preview: tensor([-0.0357])
Epoch 393, Loss: 131.0974884033203, y_pred preview: tensor([-0.0357])
Epoch 394, Loss: 131.2251739501953, y_pred preview: tensor([-0.0350])
Epoch 395, Loss: 131.0077362060547, y_pred preview: tensor([-0.0351])
Epoch 396, Loss: 130.89329528808594, y_pred preview: tensor([-0.0346])
Epoch 397, L

Epoch 510, Loss: 128.5845184326172, y_pred preview: tensor([-0.0304])
Epoch 511, Loss: 128.31561279296875, y_pred preview: tensor([-0.0294])
Epoch 512, Loss: 128.28469848632812, y_pred preview: tensor([-0.0289])
Epoch 513, Loss: 128.40211486816406, y_pred preview: tensor([-0.0293])
Epoch 514, Loss: 128.29220581054688, y_pred preview: tensor([-0.0290])
Epoch 515, Loss: 128.34097290039062, y_pred preview: tensor([-0.0299])
Epoch 516, Loss: 128.34092712402344, y_pred preview: tensor([-0.0294])
Epoch 517, Loss: 128.18310546875, y_pred preview: tensor([-0.0296])
Epoch 518, Loss: 128.1662139892578, y_pred preview: tensor([-0.0290])
Epoch 519, Loss: 128.3824462890625, y_pred preview: tensor([-0.0283])
Epoch 520, Loss: 128.38909912109375, y_pred preview: tensor([-0.0292])
Epoch 521, Loss: 128.1658477783203, y_pred preview: tensor([-0.0290])
Epoch 522, Loss: 128.40293884277344, y_pred preview: tensor([-0.0292])
Epoch 523, Loss: 128.32803344726562, y_pred preview: tensor([-0.0304])
Epoch 524, Lo

Epoch 636, Loss: 127.41291046142578, y_pred preview: tensor([-0.0247])
Epoch 637, Loss: 127.34081268310547, y_pred preview: tensor([-0.0240])
Epoch 638, Loss: 127.24725341796875, y_pred preview: tensor([-0.0237])
Epoch 639, Loss: 127.26736450195312, y_pred preview: tensor([-0.0235])
Epoch 640, Loss: 127.36961364746094, y_pred preview: tensor([-0.0231])
Epoch 641, Loss: 127.32269287109375, y_pred preview: tensor([-0.0235])
Epoch 642, Loss: 127.22699737548828, y_pred preview: tensor([-0.0233])
Epoch 643, Loss: 127.21996307373047, y_pred preview: tensor([-0.0234])
Epoch 644, Loss: 127.2890853881836, y_pred preview: tensor([-0.0237])
Epoch 645, Loss: 127.27751922607422, y_pred preview: tensor([-0.0236])
Epoch 646, Loss: 127.21560668945312, y_pred preview: tensor([-0.0240])
Epoch 647, Loss: 127.20689392089844, y_pred preview: tensor([-0.0240])
Epoch 648, Loss: 127.22820281982422, y_pred preview: tensor([-0.0241])
Epoch 649, Loss: 127.25611114501953, y_pred preview: tensor([-0.0236])
Epoch 6

Epoch 761, Loss: 127.31112670898438, y_pred preview: tensor([-0.0209])
Epoch 762, Loss: 126.80797576904297, y_pred preview: tensor([-0.0204])
Epoch 763, Loss: 127.02664184570312, y_pred preview: tensor([-0.0204])
Epoch 764, Loss: 126.8125228881836, y_pred preview: tensor([-0.0211])
Epoch 765, Loss: 126.80919647216797, y_pred preview: tensor([-0.0212])
Epoch 766, Loss: 126.93448638916016, y_pred preview: tensor([-0.0207])
Epoch 767, Loss: 126.78675079345703, y_pred preview: tensor([-0.0207])
Epoch 768, Loss: 126.8953628540039, y_pred preview: tensor([-0.0210])
Epoch 769, Loss: 126.74079132080078, y_pred preview: tensor([-0.0208])
Epoch 770, Loss: 126.89434814453125, y_pred preview: tensor([-0.0206])
Epoch 771, Loss: 126.7594223022461, y_pred preview: tensor([-0.0210])
Epoch 772, Loss: 126.7589340209961, y_pred preview: tensor([-0.0209])
Epoch 773, Loss: 126.84690856933594, y_pred preview: tensor([-0.0202])
Epoch 774, Loss: 126.79151916503906, y_pred preview: tensor([-0.0202])
Epoch 775,

Epoch 883, Loss: 126.35861206054688, y_pred preview: tensor([-0.0155])
Epoch 884, Loss: 126.42510223388672, y_pred preview: tensor([-0.0161])
Epoch 885, Loss: 126.52686309814453, y_pred preview: tensor([-0.0161])
Epoch 886, Loss: 126.33287048339844, y_pred preview: tensor([-0.0169])
Epoch 887, Loss: 126.33562469482422, y_pred preview: tensor([-0.0173])
Epoch 888, Loss: 126.51040649414062, y_pred preview: tensor([-0.0173])
Epoch 889, Loss: 126.39791107177734, y_pred preview: tensor([-0.0176])
Epoch 890, Loss: 126.32086944580078, y_pred preview: tensor([-0.0174])
Epoch 891, Loss: 126.38908386230469, y_pred preview: tensor([-0.0171])
Epoch 892, Loss: 126.52912139892578, y_pred preview: tensor([-0.0170])
Epoch 893, Loss: 126.29953002929688, y_pred preview: tensor([-0.0163])
Epoch 894, Loss: 126.43363189697266, y_pred preview: tensor([-0.0155])
Epoch 895, Loss: 126.4940185546875, y_pred preview: tensor([-0.0155])
Epoch 896, Loss: 126.35234069824219, y_pred preview: tensor([-0.0149])
Epoch 8

Train Loss: 126.05323028564453
Test Losss: 85.122802734375


(tensor(126.0532), tensor(85.1228))

In [247]:
# Train linear regression - ONLY APPROVED CLAIMS
assessor_config =  config['assessor']
assessor_config['linear_regression']['epochs'] = 1000
assessor = HomeAssessor(assessor_config)

calculate_train_val_loss(assessor, 
                         train_test_dataset_nz['train_x'], 
                         train_test_dataset_nz['train_y'],
                         train_test_dataset_nz['test_x'],
                         train_test_dataset_nz['test_y'])

Epoch 0, Loss: 5899.17138671875, y_pred preview: tensor([-169.2576])
Epoch 1, Loss: 5858.10888671875, y_pred preview: tensor([-128.4329])
Epoch 2, Loss: 5817.16064453125, y_pred preview: tensor([-87.5292])
Epoch 3, Loss: 5776.2890625, y_pred preview: tensor([-46.7186])
Epoch 4, Loss: 5735.56103515625, y_pred preview: tensor([-5.9009])
Epoch 5, Loss: 5694.8759765625, y_pred preview: tensor([34.9709])
Epoch 6, Loss: 5654.30908203125, y_pred preview: tensor([75.8108])
Epoch 7, Loss: 5613.7548828125, y_pred preview: tensor([116.6288])
Epoch 8, Loss: 5573.404296875, y_pred preview: tensor([157.4267])
Epoch 9, Loss: 5533.234375, y_pred preview: tensor([198.2122])
Epoch 10, Loss: 5493.2578125, y_pred preview: tensor([238.9937])
Epoch 11, Loss: 5453.52685546875, y_pred preview: tensor([279.7852])
Epoch 12, Loss: 5414.08837890625, y_pred preview: tensor([320.5956])
Epoch 13, Loss: 5374.94482421875, y_pred preview: tensor([361.4405])
Epoch 14, Loss: 5335.88671875, y_pred preview: tensor([402.335

Epoch 130, Loss: 2295.523681640625, y_pred preview: tensor([5976.9385])
Epoch 131, Loss: 2289.039794921875, y_pred preview: tensor([5945.9800])
Epoch 132, Loss: 2282.6083984375, y_pred preview: tensor([5913.3096])
Epoch 133, Loss: 2276.19970703125, y_pred preview: tensor([5879.0610])
Epoch 134, Loss: 2269.8525390625, y_pred preview: tensor([5843.3423])
Epoch 135, Loss: 2263.548583984375, y_pred preview: tensor([5806.2603])
Epoch 136, Loss: 2257.252197265625, y_pred preview: tensor([5767.8887])
Epoch 137, Loss: 2250.959716796875, y_pred preview: tensor([5728.2793])
Epoch 138, Loss: 2244.678955078125, y_pred preview: tensor([5687.7363])
Epoch 139, Loss: 2238.424072265625, y_pred preview: tensor([5647.1250])
Epoch 140, Loss: 2232.185791015625, y_pred preview: tensor([5605.6626])
Epoch 141, Loss: 2225.91552734375, y_pred preview: tensor([5563.4805])
Epoch 142, Loss: 2219.658447265625, y_pred preview: tensor([5520.6958])
Epoch 143, Loss: 2213.38623046875, y_pred preview: tensor([5477.4858])

Epoch 262, Loss: 1747.5164794921875, y_pred preview: tensor([2508.9187])
Epoch 263, Loss: 1746.771484375, y_pred preview: tensor([2507.6033])
Epoch 264, Loss: 1745.976318359375, y_pred preview: tensor([2504.3765])
Epoch 265, Loss: 1745.1982421875, y_pred preview: tensor([2501.4390])
Epoch 266, Loss: 1744.4239501953125, y_pred preview: tensor([2500.6123])
Epoch 267, Loss: 1743.636474609375, y_pred preview: tensor([2501.7793])
Epoch 268, Loss: 1742.87939453125, y_pred preview: tensor([2502.7339])
Epoch 269, Loss: 1742.1185302734375, y_pred preview: tensor([2502.4626])
Epoch 270, Loss: 1741.3511962890625, y_pred preview: tensor([2502.1931])
Epoch 271, Loss: 1740.59375, y_pred preview: tensor([2503.4875])
Epoch 272, Loss: 1739.8450927734375, y_pred preview: tensor([2505.7253])
Epoch 273, Loss: 1739.0802001953125, y_pred preview: tensor([2507.1589])
Epoch 274, Loss: 1738.2884521484375, y_pred preview: tensor([2507.4492])
Epoch 275, Loss: 1737.5113525390625, y_pred preview: tensor([2507.5989

Epoch 399, Loss: 1638.7342529296875, y_pred preview: tensor([2547.2053])
Epoch 400, Loss: 1637.8233642578125, y_pred preview: tensor([2537.5310])
Epoch 401, Loss: 1636.8193359375, y_pred preview: tensor([2545.0620])
Epoch 402, Loss: 1636.3070068359375, y_pred preview: tensor([2537.5012])
Epoch 403, Loss: 1635.9169921875, y_pred preview: tensor([2550.5369])
Epoch 404, Loss: 1634.778076171875, y_pred preview: tensor([2538.6833])
Epoch 405, Loss: 1633.8272705078125, y_pred preview: tensor([2544.3621])
Epoch 406, Loss: 1632.9583740234375, y_pred preview: tensor([2542.6433])
Epoch 407, Loss: 1632.20263671875, y_pred preview: tensor([2539.6653])
Epoch 408, Loss: 1631.88134765625, y_pred preview: tensor([2546.4524])
Epoch 409, Loss: 1632.4278564453125, y_pred preview: tensor([2527.3628])
Epoch 410, Loss: 1630.188232421875, y_pred preview: tensor([2544.8547])
Epoch 411, Loss: 1629.191650390625, y_pred preview: tensor([2542.7581])
Epoch 412, Loss: 1628.5777587890625, y_pred preview: tensor([253

Epoch 516, Loss: 1546.911376953125, y_pred preview: tensor([2523.4829])
Epoch 517, Loss: 1545.3580322265625, y_pred preview: tensor([2541.2668])
Epoch 518, Loss: 1543.9263916015625, y_pred preview: tensor([2533.2297])
Epoch 519, Loss: 1542.986083984375, y_pred preview: tensor([2536.3569])
Epoch 520, Loss: 1542.1771240234375, y_pred preview: tensor([2538.3696])
Epoch 521, Loss: 1541.527099609375, y_pred preview: tensor([2530.7380])
Epoch 522, Loss: 1541.1365966796875, y_pred preview: tensor([2542.9717])
Epoch 523, Loss: 1539.892822265625, y_pred preview: tensor([2529.2065])
Epoch 524, Loss: 1538.8076171875, y_pred preview: tensor([2539.2615])
Epoch 525, Loss: 1537.8009033203125, y_pred preview: tensor([2532.3032])
Epoch 526, Loss: 1536.9071044921875, y_pred preview: tensor([2538.0356])
Epoch 527, Loss: 1536.08154296875, y_pred preview: tensor([2531.0486])
Epoch 528, Loss: 1535.45654296875, y_pred preview: tensor([2540.0447])
Epoch 529, Loss: 1534.889892578125, y_pred preview: tensor([25

Epoch 645, Loss: 1419.8154296875, y_pred preview: tensor([2523.8240])
Epoch 646, Loss: 1418.7000732421875, y_pred preview: tensor([2541.0593])
Epoch 647, Loss: 1416.8582763671875, y_pred preview: tensor([2533.0999])
Epoch 648, Loss: 1416.5225830078125, y_pred preview: tensor([2523.7148])
Epoch 649, Loss: 1415.5531005859375, y_pred preview: tensor([2541.2917])
Epoch 650, Loss: 1413.591552734375, y_pred preview: tensor([2532.3086])
Epoch 651, Loss: 1413.263427734375, y_pred preview: tensor([2523.5959])
Epoch 652, Loss: 1412.3670654296875, y_pred preview: tensor([2541.6184])
Epoch 653, Loss: 1410.33544921875, y_pred preview: tensor([2532.1558])
Epoch 654, Loss: 1410.0311279296875, y_pred preview: tensor([2523.4019])
Epoch 655, Loss: 1409.0810546875, y_pred preview: tensor([2541.7778])
Epoch 656, Loss: 1406.961669921875, y_pred preview: tensor([2532.4487])
Epoch 657, Loss: 1406.482421875, y_pred preview: tensor([2522.7769])
Epoch 658, Loss: 1404.8759765625, y_pred preview: tensor([2541.426

Epoch 763, Loss: 1275.6290283203125, y_pred preview: tensor([2529.6021])
Epoch 764, Loss: 1274.3231201171875, y_pred preview: tensor([2528.1477])
Epoch 765, Loss: 1273.079833984375, y_pred preview: tensor([2529.8608])
Epoch 766, Loss: 1272.1295166015625, y_pred preview: tensor([2520.2246])
Epoch 767, Loss: 1271.620361328125, y_pred preview: tensor([2534.6750])
Epoch 768, Loss: 1270.1142578125, y_pred preview: tensor([2517.6006])
Epoch 769, Loss: 1268.2086181640625, y_pred preview: tensor([2533.7490])
Epoch 770, Loss: 1266.682861328125, y_pred preview: tensor([2528.7722])
Epoch 771, Loss: 1265.369140625, y_pred preview: tensor([2531.8044])
Epoch 772, Loss: 1264.1658935546875, y_pred preview: tensor([2534.7581])
Epoch 773, Loss: 1263.001220703125, y_pred preview: tensor([2526.5081])
Epoch 774, Loss: 1261.8209228515625, y_pred preview: tensor([2534.5654])
Epoch 775, Loss: 1260.6781005859375, y_pred preview: tensor([2521.7634])
Epoch 776, Loss: 1259.6173095703125, y_pred preview: tensor([2

Epoch 886, Loss: 1140.9659423828125, y_pred preview: tensor([2524.9438])
Epoch 887, Loss: 1139.794677734375, y_pred preview: tensor([2533.2295])
Epoch 888, Loss: 1139.2764892578125, y_pred preview: tensor([2525.9751])
Epoch 889, Loss: 1140.043701171875, y_pred preview: tensor([2542.7039])
Epoch 890, Loss: 1139.5098876953125, y_pred preview: tensor([2521.0635])
Epoch 891, Loss: 1137.8582763671875, y_pred preview: tensor([2545.0859])
Epoch 892, Loss: 1136.3638916015625, y_pred preview: tensor([2535.0532])
Epoch 893, Loss: 1135.544921875, y_pred preview: tensor([2537.8433])
Epoch 894, Loss: 1135.15966796875, y_pred preview: tensor([2543.1340])
Epoch 895, Loss: 1135.6346435546875, y_pred preview: tensor([2523.5742])
Epoch 896, Loss: 1134.8935546875, y_pred preview: tensor([2544.7139])
Epoch 897, Loss: 1133.05810546875, y_pred preview: tensor([2527.1719])
Epoch 898, Loss: 1132.0926513671875, y_pred preview: tensor([2537.0327])
Epoch 899, Loss: 1131.87255859375, y_pred preview: tensor([2526.

(tensor(1076.2845), tensor(768.4049))

In [248]:
# Train nn module -- ALL CLAIMS
assessor_config =  config['assessor']
assessor_config['simple_nn']['epochs'] = 1000
assessor_config['model'] = 'simple_nn'
assessor = HomeAssessor(assessor_config)

calculate_train_val_loss(assessor, 
                         train_test_dataset['train_x'], 
                         train_test_dataset['train_y'],
                         train_test_dataset['test_x'],
                         train_test_dataset['test_y'])

Epoch 0, Loss: 305.1286315917969, y_pred preview: tensor([0.1914])
Epoch 1, Loss: 301.7615661621094, y_pred preview: tensor([0.1932])
Epoch 2, Loss: 298.51165771484375, y_pred preview: tensor([0.1951])
Epoch 3, Loss: 295.32611083984375, y_pred preview: tensor([0.1969])
Epoch 4, Loss: 292.394775390625, y_pred preview: tensor([0.1986])
Epoch 5, Loss: 290.2262268066406, y_pred preview: tensor([0.1989])
Epoch 6, Loss: 288.1341857910156, y_pred preview: tensor([0.1987])
Epoch 7, Loss: 286.1639099121094, y_pred preview: tensor([0.1983])
Epoch 8, Loss: 284.32293701171875, y_pred preview: tensor([0.1977])
Epoch 9, Loss: 282.5472106933594, y_pred preview: tensor([0.1969])
Epoch 10, Loss: 280.80718994140625, y_pred preview: tensor([0.1960])
Epoch 11, Loss: 279.09869384765625, y_pred preview: tensor([0.1949])
Epoch 12, Loss: 277.4073486328125, y_pred preview: tensor([0.1938])
Epoch 13, Loss: 275.7178955078125, y_pred preview: tensor([0.1925])
Epoch 14, Loss: 274.0350341796875, y_pred preview: ten

Epoch 122, Loss: 205.69277954101562, y_pred preview: tensor([-0.0491])
Epoch 123, Loss: 204.94790649414062, y_pred preview: tensor([-0.0502])
Epoch 124, Loss: 204.21974182128906, y_pred preview: tensor([-0.0514])
Epoch 125, Loss: 203.4639434814453, y_pred preview: tensor([-0.0523])
Epoch 126, Loss: 202.70306396484375, y_pred preview: tensor([-0.0534])
Epoch 127, Loss: 201.93714904785156, y_pred preview: tensor([-0.0544])
Epoch 128, Loss: 201.1551513671875, y_pred preview: tensor([-0.0555])
Epoch 129, Loss: 200.3708038330078, y_pred preview: tensor([-0.0565])
Epoch 130, Loss: 199.58148193359375, y_pred preview: tensor([-0.0574])
Epoch 131, Loss: 198.78451538085938, y_pred preview: tensor([-0.0581])
Epoch 132, Loss: 198.00119018554688, y_pred preview: tensor([-0.0588])
Epoch 133, Loss: 197.1856231689453, y_pred preview: tensor([-0.0596])
Epoch 134, Loss: 196.3849639892578, y_pred preview: tensor([-0.0604])
Epoch 135, Loss: 195.56776428222656, y_pred preview: tensor([-0.0610])
Epoch 136, 

Epoch 245, Loss: 164.45973205566406, y_pred preview: tensor([-0.0428])
Epoch 246, Loss: 164.3214874267578, y_pred preview: tensor([-0.0429])
Epoch 247, Loss: 164.07763671875, y_pred preview: tensor([-0.0432])
Epoch 248, Loss: 163.9480743408203, y_pred preview: tensor([-0.0434])
Epoch 249, Loss: 163.76608276367188, y_pred preview: tensor([-0.0432])
Epoch 250, Loss: 163.586181640625, y_pred preview: tensor([-0.0432])
Epoch 251, Loss: 163.470458984375, y_pred preview: tensor([-0.0435])
Epoch 252, Loss: 163.23472595214844, y_pred preview: tensor([-0.0433])
Epoch 253, Loss: 163.17897033691406, y_pred preview: tensor([-0.0431])
Epoch 254, Loss: 162.9430694580078, y_pred preview: tensor([-0.0432])
Epoch 255, Loss: 162.84921264648438, y_pred preview: tensor([-0.0436])
Epoch 256, Loss: 162.67306518554688, y_pred preview: tensor([-0.0437])
Epoch 257, Loss: 162.418212890625, y_pred preview: tensor([-0.0435])
Epoch 258, Loss: 162.28909301757812, y_pred preview: tensor([-0.0435])
Epoch 259, Loss: 1

Epoch 362, Loss: 140.78253173828125, y_pred preview: tensor([-0.0312])
Epoch 363, Loss: 140.59571838378906, y_pred preview: tensor([-0.0313])
Epoch 364, Loss: 140.3441619873047, y_pred preview: tensor([-0.0313])
Epoch 365, Loss: 140.2410430908203, y_pred preview: tensor([-0.0312])
Epoch 366, Loss: 139.94161987304688, y_pred preview: tensor([-0.0315])
Epoch 367, Loss: 139.71888732910156, y_pred preview: tensor([-0.0315])
Epoch 368, Loss: 139.5355224609375, y_pred preview: tensor([-0.0315])
Epoch 369, Loss: 139.28395080566406, y_pred preview: tensor([-0.0316])
Epoch 370, Loss: 139.0579071044922, y_pred preview: tensor([-0.0317])
Epoch 371, Loss: 138.90200805664062, y_pred preview: tensor([-0.0316])
Epoch 372, Loss: 138.67196655273438, y_pred preview: tensor([-0.0317])
Epoch 373, Loss: 138.48580932617188, y_pred preview: tensor([-0.0316])
Epoch 374, Loss: 138.3194580078125, y_pred preview: tensor([-0.0314])
Epoch 375, Loss: 138.12692260742188, y_pred preview: tensor([-0.0314])
Epoch 376, 

Epoch 490, Loss: 130.08184814453125, y_pred preview: tensor([-0.0315])
Epoch 491, Loss: 130.25950622558594, y_pred preview: tensor([-0.0315])
Epoch 492, Loss: 129.93995666503906, y_pred preview: tensor([-0.0312])
Epoch 493, Loss: 129.99252319335938, y_pred preview: tensor([-0.0308])
Epoch 494, Loss: 130.2797088623047, y_pred preview: tensor([-0.0306])
Epoch 495, Loss: 129.98695373535156, y_pred preview: tensor([-0.0301])
Epoch 496, Loss: 130.5078582763672, y_pred preview: tensor([-0.0298])
Epoch 497, Loss: 129.90499877929688, y_pred preview: tensor([-0.0300])
Epoch 498, Loss: 130.6181182861328, y_pred preview: tensor([-0.0304])
Epoch 499, Loss: 130.43417358398438, y_pred preview: tensor([-0.0305])
Epoch 500, Loss: 130.04135131835938, y_pred preview: tensor([-0.0303])
Epoch 501, Loss: 129.93673706054688, y_pred preview: tensor([-0.0305])
Epoch 502, Loss: 130.30101013183594, y_pred preview: tensor([-0.0308])
Epoch 503, Loss: 130.06373596191406, y_pred preview: tensor([-0.0307])
Epoch 504

Epoch 614, Loss: 128.31626892089844, y_pred preview: tensor([-0.0235])
Epoch 615, Loss: 128.43453979492188, y_pred preview: tensor([-0.0234])
Epoch 616, Loss: 128.2938995361328, y_pred preview: tensor([-0.0230])
Epoch 617, Loss: 128.28717041015625, y_pred preview: tensor([-0.0228])
Epoch 618, Loss: 128.37136840820312, y_pred preview: tensor([-0.0229])
Epoch 619, Loss: 128.32176208496094, y_pred preview: tensor([-0.0230])
Epoch 620, Loss: 128.35409545898438, y_pred preview: tensor([-0.0233])
Epoch 621, Loss: 128.24169921875, y_pred preview: tensor([-0.0234])
Epoch 622, Loss: 128.2176513671875, y_pred preview: tensor([-0.0234])
Epoch 623, Loss: 128.24644470214844, y_pred preview: tensor([-0.0234])
Epoch 624, Loss: 128.34413146972656, y_pred preview: tensor([-0.0231])
Epoch 625, Loss: 128.30833435058594, y_pred preview: tensor([-0.0230])
Epoch 626, Loss: 128.2252197265625, y_pred preview: tensor([-0.0226])
Epoch 627, Loss: 128.1931610107422, y_pred preview: tensor([-0.0225])
Epoch 628, Lo

Epoch 736, Loss: 127.95930480957031, y_pred preview: tensor([-0.0177])
Epoch 737, Loss: 127.80313110351562, y_pred preview: tensor([-0.0175])
Epoch 738, Loss: 127.84269714355469, y_pred preview: tensor([-0.0178])
Epoch 739, Loss: 127.86187744140625, y_pred preview: tensor([-0.0177])
Epoch 740, Loss: 127.60466003417969, y_pred preview: tensor([-0.0173])
Epoch 741, Loss: 127.71527862548828, y_pred preview: tensor([-0.0173])
Epoch 742, Loss: 127.71371459960938, y_pred preview: tensor([-0.0178])
Epoch 743, Loss: 127.74822235107422, y_pred preview: tensor([-0.0179])
Epoch 744, Loss: 127.57881164550781, y_pred preview: tensor([-0.0178])
Epoch 745, Loss: 127.52399444580078, y_pred preview: tensor([-0.0177])
Epoch 746, Loss: 127.60951232910156, y_pred preview: tensor([-0.0175])
Epoch 747, Loss: 127.47179412841797, y_pred preview: tensor([-0.0171])
Epoch 748, Loss: 127.54430389404297, y_pred preview: tensor([-0.0169])
Epoch 749, Loss: 127.52061462402344, y_pred preview: tensor([-0.0171])
Epoch 

Epoch 854, Loss: 127.0110092163086, y_pred preview: tensor([-0.0129])
Epoch 855, Loss: 127.64075469970703, y_pred preview: tensor([-0.0122])
Epoch 856, Loss: 127.33445739746094, y_pred preview: tensor([-0.0123])
Epoch 857, Loss: 127.50582122802734, y_pred preview: tensor([-0.0129])
Epoch 858, Loss: 127.7242660522461, y_pred preview: tensor([-0.0130])
Epoch 859, Loss: 127.17701721191406, y_pred preview: tensor([-0.0128])
Epoch 860, Loss: 127.70088958740234, y_pred preview: tensor([-0.0124])
Epoch 861, Loss: 127.6727523803711, y_pred preview: tensor([-0.0126])
Epoch 862, Loss: 127.08126068115234, y_pred preview: tensor([-0.0136])
Epoch 863, Loss: 127.36404418945312, y_pred preview: tensor([-0.0141])
Epoch 864, Loss: 127.01521301269531, y_pred preview: tensor([-0.0142])
Epoch 865, Loss: 127.36881256103516, y_pred preview: tensor([-0.0140])
Epoch 866, Loss: 127.07955932617188, y_pred preview: tensor([-0.0144])
Epoch 867, Loss: 127.46234893798828, y_pred preview: tensor([-0.0146])
Epoch 868

Epoch 977, Loss: 126.70439910888672, y_pred preview: tensor([-0.0122])
Epoch 978, Loss: 127.0976791381836, y_pred preview: tensor([-0.0108])
Epoch 979, Loss: 126.66168212890625, y_pred preview: tensor([-0.0096])
Epoch 980, Loss: 127.20348358154297, y_pred preview: tensor([-0.0090])
Epoch 981, Loss: 127.2342529296875, y_pred preview: tensor([-0.0081])
Epoch 982, Loss: 126.68806457519531, y_pred preview: tensor([-0.0069])
Epoch 983, Loss: 127.28010559082031, y_pred preview: tensor([-0.0064])
Epoch 984, Loss: 126.97998809814453, y_pred preview: tensor([-0.0068])
Epoch 985, Loss: 127.08378601074219, y_pred preview: tensor([-0.0080])
Epoch 986, Loss: 127.33201599121094, y_pred preview: tensor([-0.0088])
Epoch 987, Loss: 126.89727020263672, y_pred preview: tensor([-0.0091])
Epoch 988, Loss: 127.07606506347656, y_pred preview: tensor([-0.0091])
Epoch 989, Loss: 127.07160949707031, y_pred preview: tensor([-0.0092])
Epoch 990, Loss: 126.81074523925781, y_pred preview: tensor([-0.0101])
Epoch 99

(tensor(127.1251), tensor(85.8370))

In [249]:
# Train nn module -- ALL CLAIMS
assessor_config =  config['assessor']
assessor_config['simple_nn']['epochs'] = 1000
assessor_config['model'] = 'simple_nn'
assessor = HomeAssessor(assessor_config)

calculate_train_val_loss(assessor, 
                         train_test_dataset['train_x'], 
                         train_test_dataset['train_y'],
                         train_test_dataset['test_x'],
                         train_test_dataset['test_y'])

Epoch 0, Loss: 259.1283874511719, y_pred preview: tensor([0.0316])
Epoch 1, Loss: 257.547119140625, y_pred preview: tensor([0.0320])
Epoch 2, Loss: 256.0423278808594, y_pred preview: tensor([0.0321])
Epoch 3, Loss: 254.56570434570312, y_pred preview: tensor([0.0317])
Epoch 4, Loss: 253.5424346923828, y_pred preview: tensor([0.0301])
Epoch 5, Loss: 252.54217529296875, y_pred preview: tensor([0.0274])
Epoch 6, Loss: 251.5598907470703, y_pred preview: tensor([0.0242])
Epoch 7, Loss: 250.5885467529297, y_pred preview: tensor([0.0206])
Epoch 8, Loss: 249.58056640625, y_pred preview: tensor([0.0167])
Epoch 9, Loss: 248.54209899902344, y_pred preview: tensor([0.0126])
Epoch 10, Loss: 247.4783172607422, y_pred preview: tensor([0.0085])
Epoch 11, Loss: 246.40020751953125, y_pred preview: tensor([0.0043])
Epoch 12, Loss: 245.77626037597656, y_pred preview: tensor([0.0001])
Epoch 13, Loss: 245.68479919433594, y_pred preview: tensor([-0.0043])
Epoch 14, Loss: 245.51950073242188, y_pred preview: te

Epoch 129, Loss: 175.81893920898438, y_pred preview: tensor([-0.0463])
Epoch 130, Loss: 175.21470642089844, y_pred preview: tensor([-0.0446])
Epoch 131, Loss: 174.65635681152344, y_pred preview: tensor([-0.0428])
Epoch 132, Loss: 174.0929412841797, y_pred preview: tensor([-0.0415])
Epoch 133, Loss: 173.55003356933594, y_pred preview: tensor([-0.0397])
Epoch 134, Loss: 173.03724670410156, y_pred preview: tensor([-0.0386])
Epoch 135, Loss: 172.57676696777344, y_pred preview: tensor([-0.0377])
Epoch 136, Loss: 172.11119079589844, y_pred preview: tensor([-0.0371])
Epoch 137, Loss: 171.6593017578125, y_pred preview: tensor([-0.0369])
Epoch 138, Loss: 171.2504119873047, y_pred preview: tensor([-0.0367])
Epoch 139, Loss: 170.8474578857422, y_pred preview: tensor([-0.0369])
Epoch 140, Loss: 170.44741821289062, y_pred preview: tensor([-0.0375])
Epoch 141, Loss: 170.04884338378906, y_pred preview: tensor([-0.0382])
Epoch 142, Loss: 169.65118408203125, y_pred preview: tensor([-0.0394])
Epoch 143,

Epoch 247, Loss: 143.4317169189453, y_pred preview: tensor([-0.0358])
Epoch 248, Loss: 143.1917724609375, y_pred preview: tensor([-0.0356])
Epoch 249, Loss: 142.96229553222656, y_pred preview: tensor([-0.0355])
Epoch 250, Loss: 142.77207946777344, y_pred preview: tensor([-0.0351])
Epoch 251, Loss: 142.55760192871094, y_pred preview: tensor([-0.0356])
Epoch 252, Loss: 142.27230834960938, y_pred preview: tensor([-0.0353])
Epoch 253, Loss: 142.09857177734375, y_pred preview: tensor([-0.0349])
Epoch 254, Loss: 141.89088439941406, y_pred preview: tensor([-0.0353])
Epoch 255, Loss: 141.60736083984375, y_pred preview: tensor([-0.0349])
Epoch 256, Loss: 141.4226531982422, y_pred preview: tensor([-0.0344])
Epoch 257, Loss: 141.16744995117188, y_pred preview: tensor([-0.0348])
Epoch 258, Loss: 140.92691040039062, y_pred preview: tensor([-0.0345])
Epoch 259, Loss: 140.8042755126953, y_pred preview: tensor([-0.0339])
Epoch 260, Loss: 140.46298217773438, y_pred preview: tensor([-0.0343])
Epoch 261,

Epoch 365, Loss: 129.53858947753906, y_pred preview: tensor([-0.0239])
Epoch 366, Loss: 129.5314483642578, y_pred preview: tensor([-0.0234])
Epoch 367, Loss: 129.5288543701172, y_pred preview: tensor([-0.0233])
Epoch 368, Loss: 129.52096557617188, y_pred preview: tensor([-0.0228])
Epoch 369, Loss: 129.41734313964844, y_pred preview: tensor([-0.0223])
Epoch 370, Loss: 129.40904235839844, y_pred preview: tensor([-0.0218])
Epoch 371, Loss: 129.63034057617188, y_pred preview: tensor([-0.0210])
Epoch 372, Loss: 129.41766357421875, y_pred preview: tensor([-0.0210])
Epoch 373, Loss: 129.32296752929688, y_pred preview: tensor([-0.0205])
Epoch 374, Loss: 129.35284423828125, y_pred preview: tensor([-0.0201])
Epoch 375, Loss: 129.35891723632812, y_pred preview: tensor([-0.0202])
Epoch 376, Loss: 129.3760528564453, y_pred preview: tensor([-0.0199])
Epoch 377, Loss: 129.24107360839844, y_pred preview: tensor([-0.0202])
Epoch 378, Loss: 129.2078399658203, y_pred preview: tensor([-0.0204])
Epoch 379,

Epoch 490, Loss: 128.67811584472656, y_pred preview: tensor([-0.0128])
Epoch 491, Loss: 127.84782409667969, y_pred preview: tensor([-0.0119])
Epoch 492, Loss: 128.70652770996094, y_pred preview: tensor([-0.0108])
Epoch 493, Loss: 128.3729705810547, y_pred preview: tensor([-0.0106])
Epoch 494, Loss: 128.16651916503906, y_pred preview: tensor([-0.0113])
Epoch 495, Loss: 128.46910095214844, y_pred preview: tensor([-0.0114])
Epoch 496, Loss: 127.88648986816406, y_pred preview: tensor([-0.0108])
Epoch 497, Loss: 128.49378967285156, y_pred preview: tensor([-0.0104])
Epoch 498, Loss: 128.27406311035156, y_pred preview: tensor([-0.0109])
Epoch 499, Loss: 128.01744079589844, y_pred preview: tensor([-0.0122])
Epoch 500, Loss: 128.20794677734375, y_pred preview: tensor([-0.0127])
Epoch 501, Loss: 127.80137634277344, y_pred preview: tensor([-0.0125])
Epoch 502, Loss: 128.13278198242188, y_pred preview: tensor([-0.0121])
Epoch 503, Loss: 127.78203582763672, y_pred preview: tensor([-0.0124])
Epoch 5

Epoch 609, Loss: 127.1590576171875, y_pred preview: tensor([-0.0091])
Epoch 610, Loss: 127.10528564453125, y_pred preview: tensor([-0.0085])
Epoch 611, Loss: 127.07662200927734, y_pred preview: tensor([-0.0078])
Epoch 612, Loss: 127.0924072265625, y_pred preview: tensor([-0.0071])
Epoch 613, Loss: 127.17762756347656, y_pred preview: tensor([-0.0063])
Epoch 614, Loss: 127.22315979003906, y_pred preview: tensor([-0.0062])
Epoch 615, Loss: 127.07909393310547, y_pred preview: tensor([-0.0061])
Epoch 616, Loss: 127.10238647460938, y_pred preview: tensor([-0.0063])
Epoch 617, Loss: 127.25865173339844, y_pred preview: tensor([-0.0071])
Epoch 618, Loss: 127.14311981201172, y_pred preview: tensor([-0.0073])
Epoch 619, Loss: 127.41869354248047, y_pred preview: tensor([-0.0073])
Epoch 620, Loss: 127.06712341308594, y_pred preview: tensor([-0.0077])
Epoch 621, Loss: 127.39098358154297, y_pred preview: tensor([-0.0078])
Epoch 622, Loss: 127.09883117675781, y_pred preview: tensor([-0.0073])
Epoch 62

Epoch 726, Loss: 126.71926879882812, y_pred preview: tensor([-0.0032])
Epoch 727, Loss: 126.64971160888672, y_pred preview: tensor([-0.0028])
Epoch 728, Loss: 126.67619323730469, y_pred preview: tensor([-0.0025])
Epoch 729, Loss: 126.77017211914062, y_pred preview: tensor([-0.0028])
Epoch 730, Loss: 126.6269302368164, y_pred preview: tensor([-0.0026])
Epoch 731, Loss: 126.77017974853516, y_pred preview: tensor([-0.0023])
Epoch 732, Loss: 126.76106262207031, y_pred preview: tensor([-0.0028])
Epoch 733, Loss: 126.63116455078125, y_pred preview: tensor([-0.0028])
Epoch 734, Loss: 126.96587371826172, y_pred preview: tensor([-0.0025])
Epoch 735, Loss: 126.63166046142578, y_pred preview: tensor([-0.0030])
Epoch 736, Loss: 126.91190338134766, y_pred preview: tensor([-0.0033])
Epoch 737, Loss: 126.65735626220703, y_pred preview: tensor([-0.0030])
Epoch 738, Loss: 127.14384460449219, y_pred preview: tensor([-0.0025])
Epoch 739, Loss: 126.76988220214844, y_pred preview: tensor([-0.0030])
Epoch 7

Epoch 845, Loss: 127.0324935913086, y_pred preview: tensor([-0.0032])
Epoch 846, Loss: 126.43099975585938, y_pred preview: tensor([-0.0043])
Epoch 847, Loss: 126.96068572998047, y_pred preview: tensor([-0.0045])
Epoch 848, Loss: 126.88328552246094, y_pred preview: tensor([-0.0036])
Epoch 849, Loss: 126.46170043945312, y_pred preview: tensor([-0.0011])
Epoch 850, Loss: 126.81069946289062, y_pred preview: tensor([0.0010])
Epoch 851, Loss: 126.4533920288086, y_pred preview: tensor([0.0022])
Epoch 852, Loss: 126.6640853881836, y_pred preview: tensor([0.0025])
Epoch 853, Loss: 126.74760437011719, y_pred preview: tensor([0.0028])
Epoch 854, Loss: 126.4578857421875, y_pred preview: tensor([0.0022])
Epoch 855, Loss: 126.76654052734375, y_pred preview: tensor([0.0004])
Epoch 856, Loss: 126.48570251464844, y_pred preview: tensor([-0.0026])
Epoch 857, Loss: 126.64319610595703, y_pred preview: tensor([-0.0061])
Epoch 858, Loss: 126.66590118408203, y_pred preview: tensor([-0.0082])
Epoch 859, Loss:

Epoch 964, Loss: 126.14814758300781, y_pred preview: tensor([0.0001])
Epoch 965, Loss: 126.22457885742188, y_pred preview: tensor([-0.0010])
Epoch 966, Loss: 126.14299774169922, y_pred preview: tensor([-0.0019])
Epoch 967, Loss: 126.22921752929688, y_pred preview: tensor([-0.0022])
Epoch 968, Loss: 126.04605102539062, y_pred preview: tensor([-0.0016])
Epoch 969, Loss: 126.2220230102539, y_pred preview: tensor([-0.0007])
Epoch 970, Loss: 126.18431091308594, y_pred preview: tensor([-0.0009])
Epoch 971, Loss: 126.18647003173828, y_pred preview: tensor([-0.0007])
Epoch 972, Loss: 126.11244201660156, y_pred preview: tensor([-0.0002])
Epoch 973, Loss: 126.0467758178711, y_pred preview: tensor([-0.0008])
Epoch 974, Loss: 126.12181854248047, y_pred preview: tensor([-0.0020])
Epoch 975, Loss: 126.11521911621094, y_pred preview: tensor([-0.0023])
Epoch 976, Loss: 126.04264068603516, y_pred preview: tensor([-0.0023])
Epoch 977, Loss: 126.12552642822266, y_pred preview: tensor([-0.0018])
Epoch 978

(tensor(126.7610), tensor(85.6209))